## Logic Based FizzBuzz Function [Software 1.0]

In [1]:
import pandas as pd

def fizzbuzz(n):
    
    # Logic Explanation
    if n % 3 == 0 and n % 5 == 0:
        return 'fizzbuzz'
    elif n % 3 == 0:
        return 'fizz'
    elif n % 5 == 0:
        return 'buzz'
    else:
        return 'other'

## Create Training and Testing Datasets in CSV Format

In [2]:
def createInputCSV(start,end,filename):
    
    # Why list in Python?
    inputData   = []
    outputData  = []
    
    # Why do we need training Data?
    for i in range(start,end):
        inputData.append(i)
        outputData.append(fizzbuzz(i))
    
    # Why Dataframe?
    dataset = {}
    dataset["input"]  = inputData
    dataset["label"] = outputData
    
    # Writing to csv
    pd.DataFrame(dataset).to_csv(filename)
    
    print(filename, "Created!")

## Processing Input and Label Data

In [3]:
def processData(dataset):
    
    # Why do we have to process?
    data   = dataset['input'].values
    labels = dataset['label'].values
    
    processedData  = encodeData(data)
    processedLabel = encodeLabel(labels)
    
    return processedData, processedLabel

In [4]:
def encodeData(data):
    """Converts a number to its binary reporesentaiton
    """
    processedData = []
    
    for dataInstance in data:
        
        # Why do we have number 10?
        processedData.append([dataInstance >> d & 1 for d in range(10)])
    
    return np.array(processedData)

In [5]:
from keras.utils import np_utils
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def encodeLabel(labels):
    """
    """
    processedLabel = []
    
    for labelInstance in labels:
        if(labelInstance == "fizzbuzz"):
            # Fizzbuzz
            processedLabel.append([3])
        elif(labelInstance == "fizz"):
            # Fizz
            processedLabel.append([1])
        elif(labelInstance == "buzz"):
            # Buzz
            processedLabel.append([2])
        else:
            # Other
            processedLabel.append([0])

    return np_utils.to_categorical(np.array(processedLabel),4)

Using TensorFlow backend.
/Users/ameenkhan/dev/projects/Machine-Learning/1/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Model Definition

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard

import numpy as np

input_size = 10 # 2^10 >1000
drop_out = 0.2 # adjusting for overfitting
first_dense_layer_nodes  = 256 # first trainng layer, (hidden layer), hyperparameter
# if hidden layer shouldnt be too absurt, not to large (overfitting)
# not to small
second_dense_layer_nodes = 4 # output layer

# This is a Sequential model

def get_model():
    
    # Why do we need a model?
    # Why use Dense layer and then activation?
    # Why use sequential model with layers?
    model = Sequential()
    # input matrix dimensionality 900*10
    # reason : 900 input number, 10 bits of each numbers
    
    # weight of first hidden layer = 10*256
    # dimentionality of output will be 900*256
    
    # output matrix to activation function : sigmoid, relu etc
    # relu is also used in regularization
    # why not sigmoid?
    # variants fo relu, max(x, 0)

    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('relu'))
    
    # Why dropout?
    model.add(Dropout(drop_out))
    
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('softmax'))
    # Why Softmax?
    
    model.summary()
    
    # SGD? optimizers
    #     Optimizers: 
    # adagrad, adam etc.
    
    
    # Loss function
    # Cross Entropy: KL Divergence 
    # Why use categorical_crossentropy?
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# <font color='blue'>Creating Training and Testing Datafiles</font>

In [7]:
# Create datafiles
createInputCSV(101,1001,'training.csv')
createInputCSV(1,101,'testing.csv')

training.csv Created!
testing.csv Created!


# <font color='blue'>Creating Model</font>

In [8]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               2816      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total params: 3,844
Trainable params: 3,844
Non-trainable params: 0
_________________________________________________________________


# <font color = blue>Run Model</font>

In [9]:
validation_data_split = 0.2 # trying to tackle the problem of overfitting
# difference between validation to testing
# after every epoch we validate
# validation loss and training loss
# validation avoids over fitting

# cros validation: validation data keeps rotating


num_epochs = 10000 # expoeriment on this
model_batch_size = 128
# minibatch? : between the 2 approaches:
# fitting whole data into the model, 

tb_batch_size = 32
early_patience = 100

tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')

# Read Dataset
dataset = pd.read_csv('training.csv')

# Process Dataset
processedData, processedLabel = processData(dataset)
history = model.fit(processedData
                    , processedLabel
                    , validation_split=validation_data_split
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb]
                   )

Train on 720 samples, validate on 180 samples
Epoch 1/10000
720/720 [==============================] - 0s 197us/step - loss: 1.2546 - acc: 0.4875 - val_loss: 1.1682 - val_acc: 0.5333
Epoch 2/10000
720/720 [==============================] - 0s 15us/step - loss: 1.1784 - acc: 0.5333 - val_loss: 1.1526 - val_acc: 0.5333
Epoch 3/10000
720/720 [==============================] - 0s 13us/step - loss: 1.1605 - acc: 0.5333 - val_loss: 1.1483 - val_acc: 0.5333
Epoch 4/10000
720/720 [==============================] - 0s 18us/step - loss: 1.1503 - acc: 0.5333 - val_loss: 1.1488 - val_acc: 0.5333
Epoch 5/10000
720/720 [==============================] - 0s 14us/step - loss: 1.1506 - acc: 0.5333 - val_loss: 1.1484 - val_acc: 0.5333
Epoch 6/10000
720/720 [==============================] - 0s 18us/step - loss: 1.1463 - acc: 0.5333 - val_loss: 1.1466 - val_acc: 0.5333
Epoch 7/10000
720/720 [==============================] - 0s 16us/step - loss: 1.1486 - acc: 0.5333 - val_loss: 1.1468 - val_acc: 0.5333
E

720/720 [==============================] - 0s 14us/step - loss: 1.0511 - acc: 0.5417 - val_loss: 1.1216 - val_acc: 0.5333
Epoch 61/10000
720/720 [==============================] - 0s 11us/step - loss: 1.0436 - acc: 0.5361 - val_loss: 1.1202 - val_acc: 0.5333
Epoch 62/10000
720/720 [==============================] - 0s 11us/step - loss: 1.0468 - acc: 0.5417 - val_loss: 1.1200 - val_acc: 0.5333
Epoch 63/10000
720/720 [==============================] - 0s 11us/step - loss: 1.0450 - acc: 0.5417 - val_loss: 1.1200 - val_acc: 0.5333
Epoch 64/10000
720/720 [==============================] - 0s 12us/step - loss: 1.0461 - acc: 0.5444 - val_loss: 1.1200 - val_acc: 0.5333
Epoch 65/10000
720/720 [==============================] - 0s 12us/step - loss: 1.0431 - acc: 0.5403 - val_loss: 1.1208 - val_acc: 0.5222
Epoch 66/10000
720/720 [==============================] - 0s 11us/step - loss: 1.0386 - acc: 0.5403 - val_loss: 1.1188 - val_acc: 0.5333
Epoch 67/10000
720/720 [==============================] 

Epoch 120/10000
720/720 [==============================] - 0s 12us/step - loss: 0.9158 - acc: 0.6000 - val_loss: 1.0612 - val_acc: 0.5278
Epoch 121/10000
720/720 [==============================] - 0s 12us/step - loss: 0.9134 - acc: 0.5958 - val_loss: 1.0608 - val_acc: 0.5389
Epoch 122/10000
720/720 [==============================] - 0s 12us/step - loss: 0.9026 - acc: 0.5917 - val_loss: 1.0580 - val_acc: 0.5333
Epoch 123/10000
720/720 [==============================] - 0s 12us/step - loss: 0.9121 - acc: 0.5972 - val_loss: 1.0583 - val_acc: 0.5278
Epoch 124/10000
720/720 [==============================] - 0s 13us/step - loss: 0.9086 - acc: 0.6056 - val_loss: 1.0587 - val_acc: 0.5278
Epoch 125/10000
720/720 [==============================] - 0s 12us/step - loss: 0.9020 - acc: 0.6042 - val_loss: 1.0568 - val_acc: 0.5333
Epoch 126/10000
720/720 [==============================] - 0s 12us/step - loss: 0.9000 - acc: 0.5847 - val_loss: 1.0579 - val_acc: 0.5111
Epoch 127/10000
720/720 [=========

720/720 [==============================] - 0s 13us/step - loss: 0.7873 - acc: 0.6903 - val_loss: 0.9863 - val_acc: 0.5278
Epoch 180/10000
720/720 [==============================] - 0s 13us/step - loss: 0.7918 - acc: 0.6833 - val_loss: 0.9850 - val_acc: 0.5444
Epoch 181/10000
720/720 [==============================] - 0s 12us/step - loss: 0.7927 - acc: 0.6764 - val_loss: 0.9825 - val_acc: 0.5444
Epoch 182/10000
720/720 [==============================] - 0s 13us/step - loss: 0.7879 - acc: 0.6750 - val_loss: 0.9833 - val_acc: 0.5556
Epoch 183/10000
720/720 [==============================] - 0s 12us/step - loss: 0.7722 - acc: 0.6972 - val_loss: 0.9809 - val_acc: 0.5389
Epoch 184/10000
720/720 [==============================] - 0s 12us/step - loss: 0.7819 - acc: 0.6736 - val_loss: 0.9791 - val_acc: 0.5556
Epoch 185/10000
720/720 [==============================] - 0s 12us/step - loss: 0.7673 - acc: 0.6958 - val_loss: 0.9767 - val_acc: 0.5444
Epoch 186/10000
720/720 [=========================

720/720 [==============================] - 0s 12us/step - loss: 0.6800 - acc: 0.7375 - val_loss: 0.9171 - val_acc: 0.6056
Epoch 239/10000
720/720 [==============================] - 0s 13us/step - loss: 0.6819 - acc: 0.7514 - val_loss: 0.9099 - val_acc: 0.5944
Epoch 240/10000
720/720 [==============================] - 0s 12us/step - loss: 0.6650 - acc: 0.7458 - val_loss: 0.9134 - val_acc: 0.5833
Epoch 241/10000
720/720 [==============================] - 0s 12us/step - loss: 0.6803 - acc: 0.7375 - val_loss: 0.9087 - val_acc: 0.5833
Epoch 242/10000
720/720 [==============================] - 0s 13us/step - loss: 0.6740 - acc: 0.7542 - val_loss: 0.9094 - val_acc: 0.6056
Epoch 243/10000
720/720 [==============================] - 0s 12us/step - loss: 0.6764 - acc: 0.7625 - val_loss: 0.9064 - val_acc: 0.5889
Epoch 244/10000
720/720 [==============================] - 0s 12us/step - loss: 0.6829 - acc: 0.7403 - val_loss: 0.9102 - val_acc: 0.6056
Epoch 245/10000
720/720 [=========================

...

720/720 [==============================] - 0s 12us/step - loss: 0.3431 - acc: 0.8750 - val_loss: 0.6652 - val_acc: 0.7333
Epoch 829/10000
720/720 [==============================] - 0s 14us/step - loss: 0.3150 - acc: 0.8944 - val_loss: 0.6443 - val_acc: 0.7611
Epoch 830/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3332 - acc: 0.8833 - val_loss: 0.6399 - val_acc: 0.7667
Epoch 831/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3422 - acc: 0.8903 - val_loss: 0.6605 - val_acc: 0.7278
Epoch 832/10000
720/720 [==============================] - 0s 14us/step - loss: 0.3361 - acc: 0.8944 - val_loss: 0.6407 - val_acc: 0.7556
Epoch 833/10000
720/720 [==============================] - 0s 12us/step - loss: 0.3426 - acc: 0.8722 - val_loss: 0.6538 - val_acc: 0.7333
Epoch 834/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3228 - acc: 0.8861 - val_loss: 0.6509 - val_acc: 0.7556
Epoch 835/10000
720/720 [=========================

720/720 [==============================] - 0s 12us/step - loss: 0.3454 - acc: 0.8667 - val_loss: 0.6285 - val_acc: 0.7833
Epoch 888/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3152 - acc: 0.9000 - val_loss: 0.6571 - val_acc: 0.7333
Epoch 889/10000
720/720 [==============================] - 0s 13us/step - loss: 0.2806 - acc: 0.9111 - val_loss: 0.6468 - val_acc: 0.7611
Epoch 890/10000
720/720 [==============================] - 0s 12us/step - loss: 0.3279 - acc: 0.8764 - val_loss: 0.6430 - val_acc: 0.7667
Epoch 891/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3086 - acc: 0.8903 - val_loss: 0.6527 - val_acc: 0.7444
Epoch 892/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3002 - acc: 0.8944 - val_loss: 0.6485 - val_acc: 0.7500
Epoch 893/10000
720/720 [==============================] - 0s 12us/step - loss: 0.3193 - acc: 0.9014 - val_loss: 0.6409 - val_acc: 0.7722
Epoch 894/10000
720/720 [=========================

720/720 [==============================] - 0s 14us/step - loss: 0.3122 - acc: 0.8903 - val_loss: 0.6354 - val_acc: 0.7556
Epoch 947/10000
720/720 [==============================] - 0s 12us/step - loss: 0.3027 - acc: 0.9000 - val_loss: 0.6271 - val_acc: 0.7778
Epoch 948/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3256 - acc: 0.8792 - val_loss: 0.6496 - val_acc: 0.7611
Epoch 949/10000
720/720 [==============================] - 0s 12us/step - loss: 0.3171 - acc: 0.8917 - val_loss: 0.6297 - val_acc: 0.7667
Epoch 950/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3239 - acc: 0.8750 - val_loss: 0.6327 - val_acc: 0.7667
Epoch 951/10000
720/720 [==============================] - 0s 13us/step - loss: 0.2946 - acc: 0.8847 - val_loss: 0.6538 - val_acc: 0.7333
Epoch 952/10000
720/720 [==============================] - 0s 12us/step - loss: 0.2943 - acc: 0.8972 - val_loss: 0.6636 - val_acc: 0.7333
Epoch 953/10000
720/720 [=========================

720/720 [==============================] - 0s 12us/step - loss: 0.2891 - acc: 0.8972 - val_loss: 0.6769 - val_acc: 0.7500
Epoch 1006/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3019 - acc: 0.9028 - val_loss: 0.6376 - val_acc: 0.7611
Epoch 1007/10000
720/720 [==============================] - 0s 12us/step - loss: 0.2909 - acc: 0.8972 - val_loss: 0.6504 - val_acc: 0.7389
Epoch 1008/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3238 - acc: 0.8931 - val_loss: 0.6488 - val_acc: 0.7556
Epoch 1009/10000
720/720 [==============================] - 0s 12us/step - loss: 0.3176 - acc: 0.8917 - val_loss: 0.6400 - val_acc: 0.7500
Epoch 1010/10000
720/720 [==============================] - 0s 13us/step - loss: 0.2821 - acc: 0.9056 - val_loss: 0.6513 - val_acc: 0.7500
Epoch 1011/10000
720/720 [==============================] - 0s 13us/step - loss: 0.3088 - acc: 0.9000 - val_loss: 0.6757 - val_acc: 0.7389
Epoch 1012/10000
720/720 [==================

720/720 [==============================] - 0s 12us/step - loss: 0.2646 - acc: 0.9222 - val_loss: 0.6582 - val_acc: 0.7389
Epoch 1065/10000
720/720 [==============================] - 0s 11us/step - loss: 0.2905 - acc: 0.8806 - val_loss: 0.6589 - val_acc: 0.7556
Epoch 1066/10000
720/720 [==============================] - 0s 12us/step - loss: 0.2884 - acc: 0.9028 - val_loss: 0.6461 - val_acc: 0.7611
Epoch 1067/10000
720/720 [==============================] - 0s 11us/step - loss: 0.2919 - acc: 0.8931 - val_loss: 0.6622 - val_acc: 0.7611
Epoch 1068/10000
720/720 [==============================] - 0s 13us/step - loss: 0.2711 - acc: 0.9056 - val_loss: 0.6513 - val_acc: 0.7611
Epoch 1069/10000
720/720 [==============================] - 0s 12us/step - loss: 0.3140 - acc: 0.8986 - val_loss: 0.6561 - val_acc: 0.7444
Epoch 1070/10000
720/720 [==============================] - 0s 12us/step - loss: 0.2574 - acc: 0.9028 - val_loss: 0.6444 - val_acc: 0.7500
Epoch 1071/10000
720/720 [==================

# <font color = blue>Training and Validation Graphs</font>

In [10]:
df = pd.DataFrame(history.history)
df.plot(subplots=True, grid=True, figsize=(10,15))

array([<matplotlib.axes._subplots.AxesSubplot object at 0x1276961d0>,
      dtype=object)

# <font color = blue>Testing Accuracy [Software 2.0]</font>

In [11]:
def decodeLabel(encodedLabel):
    if encodedLabel == 0:
        return "other"
    elif encodedLabel == 1:
        return "fizz"
    elif encodedLabel == 2:
        return "buzz"
    elif encodedLabel == 3:
        return "fizzbuzz"

In [12]:
wrong   = 0
right   = 0

testData = pd.read_csv('testing.csv')

processedTestData  = encodeData(testData['input'].values)
processedTestLabel = encodeLabel(testData['label'].values)
predictedTestLabel = []

for i,j in zip(processedTestData,processedTestLabel):
    y = model.predict(np.array(i).reshape(-1,10))
    predictedTestLabel.append(decodeLabel(y.argmax()))
    
    if j.argmax() == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))

print("Testing Accuracy: " + str(right/(right+wrong)*100))

output = {}
output["input"] = testData['input']
output["label"] = testData['label']
output["predicted_label"] = predictedTestLabel

opdf = pd.DataFrame(output)
opdf.to_csv('output.csv')

Errors: 18  Correct :82
Testing Accuracy: 82.0
